In [1]:
import pandas as pd
import numpy as np
import gzip
import pyarrow
import sqlalchemy as db

# Import data. Perform first inspection: NaNs, Duplicates, etc

Gzip revenue file

In [11]:
fat = gzip.open("data/faturamento.parquet.gzip", "rb")
contents = fat.read()

# gzip refuses to work. "BadGzipFile: Not a gzipped file (b'PA')"
# so instead, we're going to try importing it directly into pandas with pd.read_parquet

BadGzipFile: Not a gzipped file (b'PA')

In [ ]:
# !pip install pyarrow

In [2]:
fat= pd.read_parquet('data/faturamento.parquet.gzip')

In [3]:
fat.head() # not sorted yet

,id_nota,sku,dia_emissao_nota,uf_entrega,quantidade,receita
0,1594,1065173,2017-06-14,AL,8,968.69
1,1003,1454883,2019-02-09,SP,4,1135.49
2,2980,1708042,2017-03-14,PB,2,807.68
3,2685,1207302,2017-11-11,MG,2,889.16
4,1889,1780862,2018-05-28,SP,3,2378.21


In [35]:
# fat.shape yields 1,500,000 files, 6 columns.
fat.info() # no NaNs.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 6 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   id_nota           1500000 non-null  int64         
 1   sku               1500000 non-null  object        
 2   dia_emissao_nota  1500000 non-null  datetime64[ns]
 3   uf_entrega        1500000 non-null  object        
 4   quantidade        1500000 non-null  int64         
 5   receita           1500000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 68.7+ MB


In [36]:
fat[fat.duplicated()] # no duplicates.

,id_nota,sku,dia_emissao_nota,uf_entrega,quantidade,receita


JSON product family file

In [4]:
family = pd.read_json('data/familiasetor.json')

In [5]:
family.head()

,sku,familia,setor
0,1052974,Cachorros,Higiene e Beleza
1,1597561,Cachorros,Higiene e Beleza
2,1704369,Cachorros,Medicina e Bem Estar
3,1785992,Gatos,Alimentos
4,1555697,Cachorros,Higiene e Beleza


In [6]:
# family.shape yields 776,914 values. 3 columns.

family.info() # no nulls



<class 'pandas.core.frame.DataFrame'>
Int64Index: 776914 entries, 0 to 776913
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   sku      776914 non-null  int64 
 1   familia  776914 non-null  object
 2   setor    776914 non-null  object
dtypes: int64(1), object(2)
memory usage: 23.7+ MB


In [49]:
family[family.duplicated()] # no duplicates

,sku,familia,setor


CSV shipping file

In [74]:
# notepad indicates utf-16 LE encoding.
# Manualy altered the header to: index | uf_entrega | dia | custo_frete

In [127]:
# ship = pd.read_csv('data/frete.csv',sep = '|', encoding='utf-16',index_col = 0)
ship = pd.read_csv('data/frete.csv',sep = '|', encoding='utf-16')

ship.head() #

,index,uf_entrega,dia,custo_frete
0,0,S P,2 0 1 8 - 0 7 - 2 8,"7 0 5 1 , 5 9 1 8"
1,1,S P,2 0 1 7 - 0 2 - 1 2,"9 1 5 1 , 4 8 3 1"
2,2,R J,2 0 1 7 - 0 6 - 1 6,"4 9 0 3 , 3 4 8 6"
3,3,P E,2 0 1 8 - 0 5 - 0 5,"1 1 5 0 , 0 8 5 8"
4,4,R J,2 0 1 7 - 0 6 - 0 6,"4 2 9 2 , 1 9 7 3"


In [128]:
ship.shape #20774 rows, 4 columns

ship.info() # no NaNs

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20773 entries, 0 to 20772
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         20773 non-null  object
 1    uf_entrega   20773 non-null  object
 2    dia          20773 non-null  object
 3    custo_frete  20773 non-null  object
dtypes: object(4)
memory usage: 649.3+ KB


In [129]:
#lets normalize the headers
ship.columns=ship.columns.str.strip()

In [130]:
# we can drop this index column
ship = ship.drop(['index'],axis=1)

In [131]:
ship[ship.duplicated()] # no duplicates

,uf_entrega,dia,custo_frete


In [132]:
# We should normalize this table. Dia to datetime64[ns] and custo_frete to float (btw what a crazy cost, 4 decimals?)

In [133]:
ship_norm = ship

In [134]:
# str.replace method will solve this issue
ship_norm['uf_entrega'] = ship_norm['uf_entrega'].str.replace(' ', '')
ship_norm['dia'] = ship_norm['dia'].str.replace(' ', '')
ship_norm['custo_frete'] = ship_norm['custo_frete'].str.replace(' ', '')


In [135]:
ship_norm.head()

,uf_entrega,dia,custo_frete
0,SP,2018-07-28,"7051,5918"
1,SP,2017-02-12,"9151,4831"
2,RJ,2017-06-16,"4903,3486"
3,PE,2018-05-05,"1150,0858"
4,RJ,2017-06-06,"4292,1973"


In [141]:
ship_norm.info()
# now change dia to date, round the custo_frete and.. change commas to dot

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20773 entries, 0 to 20772
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uf_entrega   20773 non-null  object        
 1   dia          20773 non-null  datetime64[ns]
 2   custo_frete  20773 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 487.0+ KB


In [140]:
ship_norm['dia'] = pd.to_datetime(ship_norm['dia'])

In [143]:
#now to round cost:
ship_norm['custo_frete'] = ship_norm['custo_frete'].str.replace(',','.')


In [146]:
ship_norm['custo_frete'] = ship_norm['custo_frete'].astype('float')

In [150]:
ship_norm['custo_frete'] = ship_norm['custo_frete'].round(2)

In [151]:
ship_norm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20773 entries, 0 to 20772
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   uf_entrega   20773 non-null  object        
 1   dia          20773 non-null  datetime64[ns]
 2   custo_frete  20773 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 487.0+ KB


TXT weight file. "Peso unitário do SKU. Usado para o custo de frete."

In [154]:
weight = pd.read_table('data/peso_unitario.txt',encoding='utf-16', sep = '  ', engine='python') # also has encoding utf-16.
# manually inserted a header: 'sku   peso_unitario'

weight.head()

,sku,peso_unitario
0,1 1 8 2 4 0 1,3 . 1 4 0 0 0 0
1,1 0 4 5 1 1 0,0 . 3 0 0 0 0 0
2,1 6 0 3 0 2 2,0 . 2 7 0 0 0 0
3,1 1 4 2 9 7 9,0 . 1 3 0 0 0 0
4,1 1 1 4 9 8 8,0 . 2 0 0 0 0 0


In [155]:
# weight.shape # yields 776,914 values (same as family). 2 columns.

weight.info() # no nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 776914 entries, 0 to 776913
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   sku             776914 non-null  object
 1    peso_unitario  776914 non-null  object
dtypes: object(2)
memory usage: 11.9+ MB


In [156]:
weight[weight.duplicated()] # no duplicates

,sku,peso_unitario


In [158]:
weight.columns = weight.columns.str.strip()

In [162]:
weight['sku'] = weight['sku'].str.replace(' ','')
weight['peso_unitario'] = weight['peso_unitario'].str.replace(' ','')

In [164]:
weight.tail()

,sku,peso_unitario
776909,1183894,7.630000
776910,1237125,10.200000
776911,1327455,0.150000
776912,1927355,3.110000
776913,1228822,0.100000


# SQL Ingestion with sqlalchemy

In [166]:
db_server='pymysql'
user='root'
db_port = '3306'
password = 'password'
ip = '127.0.0.1'
db_name = 'petlove' # schema do sql check
engine = db.create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')
conn = engine.connect()

In [167]:
fat.to_sql('fat', conn, if_exists='replace', index=False)

1500000

In [168]:
family.to_sql('family', conn, if_exists='replace', index=False)

776914

In [169]:
ship_norm.to_sql('ship', conn, if_exists='replace', index=False)

20773

In [170]:
weight.to_sql('weight', conn, if_exists='replace', index=False)

776914